# Marginalisation

In this notebook, we will load the data from after the second fit and perform the marginalisation.

I need to get the data import working. The cells will have to be broken up more. I need to check for astropy units.

In [ ]:
# Imports
import numpy as np
import os
import time
import sys
import matplotlib.pyplot as plt
%matplotlib inline
import astropy.units as u
from astropy.constants import G

os.chdir('../HST_python')
from config import CONFIG_INI
import margmodule as marg

## Load the data

Load the results into the notebook and make sure the data looks right. We're loading the data after the second fit.

In [ ]:
#dpath = '/Users/ilaginja/Documents/data_from_repos/hst_marg_data/outputs/LevMar_2019-5-20'
dpath = '/Users/hwakeford/Documents/GitHub/HST_Marginalization/outputs/'
fname = 'LevMar_after_2nd_fit.npz'

data = np.load(os.path.join(dpath, fname))

In [ ]:
# Investigate data
print(data.files)

In [ ]:
# Read all data columns
sys_stats = data['sys_stats']
sys_date = data['sys_date']
sys_phase = data['sys_phase']
sys_rawflux = data['sys_rawflux']
sys_rawflux_err = data['sys_rawflux_err']
sys_flux = data['sys_flux']
sys_flux_err = data['sys_flux_err']
sys_residuals = data['sys_residuals']
sys_model = data['sys_model']
sys_model_phase = data['sys_model_phase']
sys_systematic_model = data['sys_systematic_model']
sys_params = data['sys_params']
sys_depth = data['sys_depth']
sys_depth_err = data['sys_depth_err']
sys_epoch = data['sys_epoch']
sys_epoch_err = data['sys_epoch_err']
sys_evidenceAIC = data['sys_evidenceAIC']
sys_evidenceBIC = data['sys_evidenceBIC']

In [ ]:
print(sys_stats.shape)
print(sys_date.shape)
print(sys_phase.shape)
print(sys_rawflux.shape)
# ... and so on

## Sort by AIC evidence

The AIC tells us how "good" the fit is.

The evidence is the (simplified) negative log likelihood of the data begin fit by this model. Given this model, the data is likely to fit this model by *this* degree. How well can this model be described by the data we’ve given it.

We don’t know the systematics very well, those change over time and what it looked at last. The Marginalisation is because if you slightly change your observation, we don’t know which of the 50 models actually describes observation best. We calculate for each model how well it describes data. Based on those numbers we get ordering from best to worst model - make a value that tells you which one fits the data best

BIC penalizes models that are more complicated -  the more parameters you fit, the worse the model accordance with the data. Fewer free parameters are better.
AIC only penalizes you by number of free parameters and not by how much data you have as well, it's a bit more lenient, it depends on how much you trust your data to decide whether to chose BIC and AIC. Literature usually prefers AIC, although BC is usually a slightly higher uncertainty. (We also AIC here.)

In [ ]:
print('sys_evidenceAIC per model:')
print(sys_evidenceAIC.shape)
print(sys_evidenceAIC)

`a` is the sorted system evidence (AIC) array from largest to lowest.

In [ ]:
a = (np.sort(sys_evidenceAIC))[::-1]
print(a.shape)
print(a)

## Top 10 systematic models

Now we will have a closer look at the top 10 systematic models, sorted. by best AIC.

In [ ]:
print('\nTOP 10 SYSTEMATIC MODELS: AIC')
# Print the AIC for the top 10 systematic models
print(a[:10])
# Print all the AIC values (why?)
print('All AIC:')
print(sys_evidenceAIC)

## Reformat arrays - by ditching negative evidence values?

In [ ]:
# REFORMAT all arrays with just positive values
pos = np.where(sys_evidenceAIC > -500)   #TODO: change hard coded number?
if len(pos) == 0:
    pos = -1
    
npos = len(pos[0])   # NOT-REUSED
#TODO: What is getting printed here?
print('npos: {}'.format(npos))
print('POS positions = {}'.format(pos))

Now that we've gotten rid of models with AIC evidence we don't like (or something like that), we redefine the arrays. Although here, we still have ALL of the systematic models, because none of them seem to be falling out of line.

In [ ]:
count_AIC = sys_evidenceAIC[pos]

count_depth = sys_depth[pos]
count_depth_err = sys_depth_err[pos]

count_epoch = sys_epoch[pos]
count_epoch_err = sys_epoch_err[pos]

count_residuals = sys_residuals[pos]
count_date = sys_date[pos]
count_flux = sys_flux[pos]
count_flux_err = sys_flux_err[pos]
count_phase = sys_phase[pos]
count_model_y = sys_model[pos]
count_model_x = sys_model_phase[pos]

print(count_AIC.shape)

## Find most likely model

In [ ]:
beta = np.min(count_AIC)
w_q = (np.exp(count_AIC - beta)) / np.sum(np.exp(count_AIC - beta))

print('beta: {}'.format(beta))
print('w_q: {}'.format(w_q))

In [ ]:
n01 = np.where(w_q >= 0.05)
print('{} models have a weight over 0.05. -> Models: {} with weigths: {}'.format(n01[0].shape, n01, w_q[n01]))
print('Most likely model is number {} at w_q={}'.format(np.argmax(w_q), np.max(w_q)))

SDNR is the scatter on the residuals.

In [ ]:
best_sys_weight = np.argmax(w_q)   #TODO: redefined couple of lines below...
print('SDNR best model from evidence = {}, for model {}'.format(
      np.std(count_residuals[best_sys_weight,:]) / np.sqrt(2) * 1e6, best_sys_weight))

rl_sdnr = np.zeros(len(w_q))
for i in range(len(w_q)):
    rl_sdnr[i] = (np.std(count_residuals[i,:]) / np.sqrt(2)) * 1e6
best_sys_sdnr = np.argmin(rl_sdnr)

print('SDNR best model from minimization = {} for model {}'.format(np.min(rl_sdnr), best_sys_sdnr))

## Plots

We can ignore these for now, we will deal with the plots later.

In [ ]:
plt.figure(3)
plt.subplot(3,1,1)
plt.plot(w_q)
plt.title('w_q')
plt.subplot(3,1,2)
plt.plot(rl_sdnr)
plt.title('rl_sdnr')
plt.subplot(3,1,3)
plt.errorbar(np.arange(1, len(count_depth)+1), count_depth, yerr=count_depth_err, fmt='.')
plt.title('count_depth')
plt.draw()
plt.pause(0.05)

plt.figure(4)
plt.subplot(3, 1, 1)
plt.scatter(sys_phase[0,:], sys_flux[0,:])
plt.ylim(np.min(sys_flux[0,:]) - 0.001, np.max(sys_flux[0,:]) + 0.001)
plt.xlabel('sys_phase')
plt.ylabel('sys_flux')

plt.subplot(3,1,2)
plt.scatter(count_phase[best_sys,:], count_flux[best_sys,:])
plt.plot(count_model_x[best_sys,:], count_model_y[best_sys,:])
plt.ylim(np.min(count_flux[0,:]) - 0.001, np.max(count_flux[0,:]) + 0.001)
plt.xlabel('count_phase')
plt.ylabel('count_flux')

plt.subplot(3,1,3)
plt.errorbar(count_phase[best_sys,:], count_residuals[best_sys,:], yerr=count_flux_err[best_sys,:], fmt='.')
plt.ylim(-1000, 1000)
plt.xlabel('count_phase')
plt.ylabel('count_residuals')
plt.hlines(0.0, xmin=np.min(count_phase[best_sys,:]), xmax=np.max(count_phase[best_sys,:]), colors='r', linestyles='dashed')
#plt.hlines(0.0 - (rl_sdnr[best_sys] * cut_down), xmin=np.min(count_phase[best_sys,:]), xmax=np.max(count_phase[best_sys,:]), colors='r', linestyles='dotted')
#plt.hlines(0.0 + (rl_sdnr[best_sys] * cut_down), xmin=np.min(count_phase[best_sys,:]), xmax=np.max(count_phase[best_sys,:]), colors='r', linestyles='dotted')
plt.draw()
plt.show()

## Perform the marginalisation

In [ ]:
### Radius ratio - this one always gets calculated
marg_rl, marg_rl_err = marg.marginalisation(count_depth, count_depth_err, w_q)
print('Rp/R* = {} +/- {}'.format(marg_rl, marg_rl_err))

In [ ]:
### Center of transit time (epoch)
marg_epoch = None
marg_epoch_err = None
if not tmodel.epoch.frozen:
    marg_epoch, marg_epoch_err = marg.marginalisation(count_epoch, count_epoch_err, w_q)
    print('Epoch = {} +/- {}'.format(marg_epoch, marg_epoch_err))

In [ ]:
### Inclination
marg_inclin_rad = None
marg_inclin_rad_err = None
marg_inclin_deg = None
marg_inclin_deg_err = None

if not tmodel.inclin.frozen:
    # Inclication in radians
    marg_inclin_rad, marg_inclin_rad_err = marg.marginalisation(sys_params[:, 3], sys_params_err[:, 3], w_q)
    print('inc (rads) = {} +/- {}'.format(marg_inclin_rad, marg_inclin_rad_err))

    # Inclination in degrees
    conv1 = sys_params[:, 3] / (2 * np.pi / 360)
    conv2 = sys_params_err[:, 3] / (2 * np.pi / 360)
    marg_inclin_deg, marg_inclin_deg_err = marg.marginalisation(conv1, conv2, w_q)
    print('inc (deg) = {} +/- {}'.format(marg_inclin_deg, marg_inclin_deg_err))

In [ ]:
### MsMpR
marg_msmpr = None
marg_msmpr_err = None
marg_aors = None
marg_aors_err = None

if not tmodel.msmpr.frozen:
    marg_msmpr, marg_msmpr_err = marg.marginalisation(sys_params[:, 4], sys_params_err[:, 4], w_q)
    print('MsMpR = {} +/- {}'.format(marg_msmpr, marg_msmpr_err))

    # Recalculate a/R* (actually the constant for it) based on the new MsMpR value which may have been fit in the routine.
    constant1 = (G * np.square((tmodel.period.val * u.d).to(u.s)) / (4 * np.pi * np.pi)) ** (1 / 3.)   #TODO: period is constant - make pretty

    marg_aors = constant1 * (marg_msmpr ** (1./3.))
    marg_aors_err = constant1 * (marg_msmpr_err ** (1./3.)) / marg_aors
    print('a/R* = {} +/- {}'.format(marg_aors, marg_aors_err))

## Save to file

E.g. for comparison to IDL results.

In [ ]:
# np.savez(os.path.join(outDir, 'analysis_circle_G141_marginalised_'+run_name), w_q=w_q, best_sys=best_sys,
#          marg_rl=marg_rl, marg_rl_err=marg_rl_err, marg_epoch=marg_epoch, marg_epoch_err=marg_epoch_err,
#          marg_inclin_rad=marg_inclin_rad, marg_inclin_rad_err=marg_inclin_rad_err, marg_inclin_deg=marg_inclin_deg,
#          marg_inclin_deg_err=marg_inclin_deg_err, marg_msmpr=marg_msmpr, marg_msmpr_err=marg_msmpr_err,
#          marg_aors=marg_aors, marg_aors_err=marg_aors_err, rl_sdnr=rl_sdnr, pos=pos)